# 데이터 로드

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
# from sklearn.utils import resample

train_df = pd.read_feather("/content/drive/MyDrive/data_ftr/train_data.ftr")
# test_df = pd.read_feather("/content/drive/MyDrive/data_ftr/test_data.ftr")

In [2]:
# train_df.shape

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 191 entries, customer_ID to target
dtypes: category(11), datetime64[ns](1), float16(177), int64(1), object(1)
memory usage: 2.0+ GB


In [4]:
train_df = train_df.groupby('customer_ID')
train_df = train_df.tail(1)

In [5]:
train_df.shape

(458913, 191)

In [6]:
# customer_ID, S_2 컬럼 제거 (유의미 하지 않음)
train_df = train_df.drop(['customer_ID', 'S_2'], axis=1)

# 결측치 확인

In [7]:
# 각 컬럼별 결측치 확인
train_df.isnull().sum()

P_2         2969
D_39           0
B_1            0
B_2           31
R_1            0
           ...  
D_142     378598
D_143       2830
D_144          0
D_145       2830
target         0
Length: 189, dtype: int64

In [8]:
total_cols = train_df.columns.to_list()

# 수치형과 범주형 컬럼 구분
cat_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
# 전체 데이터 - cat_columns
num_columns = [col for col in total_cols if col not in cat_columns]

In [9]:
print(f'수치형 컬럼: {num_columns}')
print(f'범주형 컬럼: {cat_columns}')

수치형 컬럼: ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_29', 'S_18', 'D_86', 'D_87', 'R_17', 'R_18', 'D_88', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_22', 'R_23', 'D_91', 'D_92', 'D_93', 'D_94', 'R_24', 'R_25', 'D_96', 'S_22', 'S_23', 'S_24', 'S_25', 'S

In [10]:
num_columns_copy = num_columns.copy() # 수치형 데이터 복제
cat_columns_copy = cat_columns.copy() # 범주형 데이터 복제

## 결측치 채우기 및 LGBM

## 범주형 데이터의 결측치 확인

In [11]:
null_cat = round((train_df[cat_columns].isna().sum()/train_df.shape[0]*100),2).sort_values(ascending=False).astype(str)+('%')
null_cat = null_cat.to_frame().rename(columns={0:'Missing %'})
null_cat.head(30)

,Missing %
D_66,88.54%
D_68,1.96%
D_114,1.31%
D_116,1.31%
D_117,1.31%
D_120,1.31%
B_30,0.01%
B_38,0.01%
D_126,0.0%
D_63,0.0%


In [ ]:
# 범주형 데이터는 전체 데이터를 One-Hot Encoding 대체
for column in cat_columns:
    ohe = OneHotEncoder()
    train_df[column] = ohe.fit_transform(train_df[column].values.reshape(-1, 1)).toarray()
    print(f"{column} OneHotEncoding 완료")

In [ ]:
# 수치형 데이터 결측치 0으로 채우기
train_df[num_columns] = train_df[num_columns].fillna(0)
print("수치형 데이터 결측치 0으로 채우기 완료")

In [14]:
# train_df를 cat_columns와 num_columns를 합체
train_df = pd.concat([train_df[cat_columns_copy], train_df[num_columns_copy]], axis=1)

In [15]:
train_df.head(10)

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
12,0,1,1,0,4,0,1,2,1,1,...,0.014122,0.158936,0.007187,0.004234,0.005085,0.00000,0.005810,0.002970,0.008530,0
25,0,1,1,0,0,0,1,1,1,1,...,0.014122,0.158936,0.002981,0.007481,0.007874,0.00000,0.003284,0.003170,0.008514,0
38,0,0,1,0,0,0,1,1,2,1,...,0.014122,0.158936,0.007381,0.006622,0.000965,0.00000,0.002201,0.000834,0.003445,0
51,0,1,1,0,6,0,1,1,1,1,...,0.014122,0.158936,0.002705,0.006184,0.001899,0.00000,0.008186,0.005558,0.002983,0
64,0,0,1,0,4,0,1,1,1,0,...,0.014122,0.158936,0.002974,0.004162,0.005764,0.00000,0.008156,0.006943,0.000905,0
77,0,1,1,0,1,0,1,1,2,1,...,0.014122,0.158936,0.009399,0.007835,0.000957,0.00000,0.009056,0.006943,0.001446,0
90,0,2,0,0,2,0,0,1,2,1,...,0.014122,0.158936,0.006012,0.005222,0.000937,0.00000,0.009789,0.003704,0.006275,0
103,0,2,1,0,6,0,1,1,2,1,...,0.014122,0.158936,0.002262,0.005219,0.002153,0.00000,0.001169,0.003349,0.000675,0
116,1,6,0,0,0,0,1,1,3,0,...,0.014122,0.158936,1.000977,0.007061,0.875977,0.18457,1.004883,0.003349,0.734375,1
129,0,0,0,0,4,0,1,1,1,1,...,0.014122,0.158936,0.009811,0.002213,0.004070,0.00000,0.005428,0.007107,0.006271,1


# 모델 학습

In [16]:
X = train_df.drop('target', axis=1)
y = train_df['target']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
X_train.shape

(367130, 188)

In [18]:
X_test.shape

(91783, 188)

In [19]:
y_train.shape

(367130,)

In [20]:
y_test.shape

(91783,)

In [21]:
# LightGBM으로 모델 학습
model = LGBMClassifier(random_state=42, n_estimators=1000, learning_rate=0.01, objective='binary')
model.fit(X_train, y_train)

# pkl 추출
import pickle
with open('lgbm_model(label&mean).pkl', 'wb') as f:
    pickle.dump(model, f)

[LightGBM] [Info] Number of positive: 95285, number of negative: 271845
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.185935 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 44141
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 187
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.259540 -> initscore=-1.048360
[LightGBM] [Info] Start training from score -1.048360


In [22]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()

    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

# 예측 확률 계산
pred_prob = model.predict_proba(X_test)[:, 1]
y_pred = pd.DataFrame({'prediction': pred_prob}, index=X_test.index)
y_true = pd.DataFrame({'target': y_test}, index=X_test.index)

# 성능 평가
print("AMEX Score:", amex_metric(y_true, y_pred))

AMEX Score: 0.782429649449925


In [23]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 모델의 성능 리포트를 출력하는 함수
def generate_model_report(y_true, y_pred):
    # 평가 지표 계산
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # 결과 출력
    print("Model Performance:")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"R^2: {r2}")

# 예측 결과와 실제 타겟 데이터를 준비
y_true = y_test
y_pred = model.predict(X_test)

# 모델의 성능 리포트 출력
generate_model_report(y_true, y_pred)

Model Performance:
MSE: 0.10028000828040051
MAE: 0.10028000828040051
R^2: 0.47417886525162434


In [24]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)

# 정밀도 계산
precision = precision_score(y_test, y_pred)

# 재현율 계산
recall = recall_score(y_test, y_pred)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# 분류 보고서 출력
report = classification_report(y_test, y_pred)
print("\nXGBoost Report:\n", report)

Accuracy: 0.8997199917195995
Precision: 0.8017593333052738
Precision: 0.8017593333052738
Recall: 0.8091152359512381
F1 Score: 0.8054204896198891

XGBoost Report:
               precision    recall  f1-score   support

           0       0.93      0.93      0.93     68240
           1       0.80      0.81      0.81     23543

    accuracy                           0.90     91783
   macro avg       0.87      0.87      0.87     91783
weighted avg       0.90      0.90      0.90     91783



# test data 실험

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
# from sklearn.utils import resample

In [26]:
# test_data 로드
test_df = pd.read_feather("/content/drive/MyDrive/data_ftr/test_data.ftr")

In [27]:
test_df = test_df.groupby('customer_ID')
test_df = test_df.tail(1)
test_df.shape

(924621, 190)

In [28]:
test_df.head(10)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
8,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-10-12,0.568848,0.121399,0.010780,1.009766,0.006924,0.149414,0.000396,0.003576,...,NaN,NaN,NaN,0.005913,0.001250,0.006542,NaN,0.009163,0.003691,0.003220
21,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,2019-04-15,0.841309,0.126465,0.016556,1.008789,0.009712,0.112183,0.006191,0.011383,...,NaN,NaN,NaN,0.004345,0.000866,0.009117,NaN,0.002197,0.000247,0.007778
34,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,2019-10-16,0.697754,0.002724,0.001485,0.810059,0.002621,0.166138,0.004887,0.015945,...,NaN,NaN,NaN,1.000977,0.008896,0.895996,0.150146,1.009766,0.457764,0.092041
47,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,2019-04-22,0.513184,0.324707,0.149536,0.205688,0.002277,0.181152,0.005814,0.498535,...,NaN,NaN,NaN,1.007812,0.003754,0.919922,0.255371,1.007812,0.500977,0.182983
60,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,2019-10-22,0.254395,0.768066,0.563477,0.038025,0.502930,0.168335,0.009483,0.831055,...,NaN,NaN,NaN,0.006622,0.001140,0.009529,NaN,0.009407,0.001557,0.000525
73,00004ffe6e01e1b688170bbd108da8351bc4c316eacfef...,2019-10-04,1.003906,0.004520,0.015442,1.004883,0.007072,0.157837,0.008255,0.011215,...,NaN,NaN,NaN,0.005520,0.000525,0.007641,NaN,0.007229,0.007462,0.005432
86,00007cfcce97abfa0b4fa0647986157281d01d3ab90de9...,2019-04-27,0.351318,0.035522,0.321289,0.043304,0.507812,0.553223,0.003963,0.624512,...,NaN,NaN,NaN,1.000977,0.005283,0.878906,0.128174,1.000000,0.020203,0.183350
99,000089cc2a30dad8e6ba39126f9d86df6088c9f975093a...,2019-04-25,0.307129,0.000292,0.126343,0.445068,0.001740,0.105347,0.002327,0.079285,...,NaN,NaN,NaN,0.009155,0.001059,0.003994,NaN,0.002428,0.001195,0.009186
112,00008f50a1dd76fa211ba36a2b0d5a1b201e4134a5fd53...,2019-10-24,0.332031,0.005196,0.330078,0.055511,0.001949,0.515625,0.000360,0.362793,...,NaN,NaN,NaN,0.009781,0.004021,0.002819,NaN,0.007679,0.006760,0.004795
125,0000b48a4f27dc1d61e78d081678e811620300b88eb3ab...,2019-10-09,0.949707,0.009674,0.009193,0.812500,0.001989,NaN,0.001398,0.008179,...,NaN,NaN,NaN,0.003918,0.002836,0.006107,NaN,0.003353,0.007519,0.006737


In [29]:
# total_cols = test_df.columns.to_list()

# # 수치형과 범주형 컬럼 구분
# cat_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
# # 전체 데이터 - cat_columns
# num_columns = [col for col in total_cols if col not in cat_columns]

# # 범주형 데이터는 전체 데이터를 One-hot Encoding으로 대체
# for column in cat_columns:
#     ohe = OneHotEncoder()
#     test_df[column] = ohe.fit_transform(test_df[column].values.reshape(-1, 1)).toarray()
#     print(f"{column} OneHotEncoding 완료")

# # 수치형 데이터 결측치 처리
# test_df[num_columns] = test_df[num_columns].fillna(0)
# print("수치형 데이터 결측치 처리 완료")

# # test_df를 cat_columns와 num_columns를 합체
# test_df = pd.concat([test_df[cat_columns], test_df[num_columns]], axis=1)

In [ ]:
total_cols = test_df.columns.to_list()

# 수치형과 범주형 컬럼 구분
cat_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
# 전체 데이터 - cat_columns
num_columns = [col for col in total_cols if col not in cat_columns]

# 범주형 데이터는 전체 데이터를 One-Hot Encoding 대체
for column in cat_columns:
    ohe = OneHotEncoder()
    test_df[column] = ohe.fit_transform(test_df[column].values.reshape(-1, 1)).toarray()
    print(f"{column} OneHotEncoding 완료")

# 수치형 데이터 결측치 처리
test_df[num_columns] = test_df[num_columns].fillna(0)

# test_df를 cat_columns와 num_columns를 합체
test_df = pd.concat([test_df[cat_columns], test_df[num_columns]], axis=1)

In [ ]:
# customer_ID 따로 저장
customer_ID = test_df['customer_ID']

In [ ]:
# S_2, customer_ID 컬럼 드랍
test_df = test_df.drop(columns=['customer_ID', 'S_2'])

In [ ]:
# LGBM 모델로 예측, predict_proba
pred_prob = model.predict_proba(test_df)[:, 1]

In [ ]:
# customer_ID와 prediction을 submission.csv로 추출
submission = pd.DataFrame({'customer_ID': customer_ID, 'prediction': pred_prob})
submission.to_csv('submission_predict(LGBM&OneHot&Zero).csv', index=False)